In [1]:
import envloader 

from langchain.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain.document_loaders import WebBaseLoader, TextLoader
from langchain.embeddings import GPT4AllEmbeddings



import bs4
from langchain import hub


from langchain.chat_models import ChatOpenAI
from langchain.llms.openllm import OpenLLM
from langchain.llms.gpt4all import GPT4All


from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [2]:
text = r"D:\P2\Python\llm-proj\random.txt" 
link = "https://en.wikipedia.org/wiki/Breaking_Bad"

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)

In [4]:
#loader = TextLoader(text)

In [5]:
loader = WebBaseLoader(link)

In [6]:
help(loader.load_and_split)

Help on method load_and_split in module langchain.document_loaders.base:

load_and_split(text_splitter: Optional[langchain.text_splitter.TextSplitter] = None) -> List[langchain_core.documents.base.Document] method of langchain.document_loaders.web_base.WebBaseLoader instance
    Load Documents and split into chunks. Chunks are returned as Documents.
    
    Args:
        text_splitter: TextSplitter instance to use for splitting documents.
          Defaults to RecursiveCharacterTextSplitter.
    
    Returns:
        List of Documents.



In [7]:
splits = loader.load_and_split(text_splitter )
for index, split in enumerate(splits[:5] , 1 ):
    print(f"Split {index}:{split}\n\n")
    
len(splits)

Split 1:page_content='Breaking Bad - Wikipedia' metadata={'source': 'https://en.wikipedia.org/wiki/Breaking_Bad', 'title': 'Breaking Bad - Wikipedia', 'language': 'en', 'start_index': 4}


Split 2:page_content='Jump to content' metadata={'source': 'https://en.wikipedia.org/wiki/Breaking_Bad', 'title': 'Breaking Bad - Wikipedia', 'language': 'en', 'start_index': 65}


Split 3:page_content='Jump to content\n\n\n\n\n\n\n\nMain menu' metadata={'source': 'https://en.wikipedia.org/wiki/Breaking_Bad', 'title': 'Breaking Bad - Wikipedia', 'language': 'en', 'start_index': -1}


Split 4:page_content='Main menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide' metadata={'source': 'https://en.wikipedia.org/wiki/Breaking_Bad', 'title': 'Breaking Bad - Wikipedia', 'language': 'en', 'start_index': 88}


Split 5:page_content='Navigation' metadata={'source': 'https://en.wikipedia.org/wiki/Breaking_Bad', 'title': 'Breaking Bad - Wikipedia', 'language': 'en', 'start_index': 139}




4500

In [9]:
gpt4all_embd = GPT4AllEmbeddings()

In [10]:
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings() , )

In [11]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = GPT4All(model = r"C:\Users\obsidian\Desktop\mistral-7b-openorca.Q4_0.gguf" , )

In [12]:
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
import torch 
import gc 
torch.cuda.empty_cache() 
gc.collect()

0

In [13]:
queries = [
    #"Who is Aniruddh?", #Direct 
    "Who is Walter White?", #Not present 
    "What is capitalism?", #General/Not direct 
    #"How is Aniruddh linked with Gary Vee?", #Indirect context 
]

for index, query in enumerate(queries , 1 ):
    print(f"Query {index}: {query}")
    print(f"Result: {rag_chain.invoke(query)}\n\n")

Query 1: Who is Walter White?
Result:  Walter White is a fictional character from the television series "Breaking Bad." He was portrayed by Bryan Cranston and played a chemistry teacher who turned to a life of crime after being diagnosed with terminal cancer.


Query 2: What is capitalism?
Result:  I don't know.


